### 1. Computing Wasserstein-1 distance between Q and steady state of M/M/1 queue

In [1]:
import numpy as np
from scipy.stats import wasserstein_distance
import math

In [2]:
def W1_MM1(q: list, a: list, rho: float, padding: int = 100, tolerance: float = 1e-9):
    """
    Computes the 1-Wasserstein distance between an empirical distribution Q
    and the theoretical M/M/1 steady-state distribution X, using rho directly.

    The theoretical distribution X is truncated at a point determined by both
    a padding value beyond the empirical support and a tolerance for the
    tail mass.

    Parameters
    ----------
    q : list or np.ndarray
        Probability weights of the empirical distribution Q.
    a : list or np.ndarray
        Support points of Q, corresponding to q.
    rho : float
        The traffic intensity (lambda / mu) of the M/M/1 queue. Must be in [0, 1).
    padding : int, optional
        The support for the calculation will extend at least this many points
        beyond the maximum support point of Q. Default is 10.
    tolerance : float, optional
        The tail mass of the truncated theoretical distribution will be less
        than this value. Default is 1e-9.

    Returns
    -------
    float
        The computed 1-Wasserstein distance.
    """
    # --- 1. Input Validation and Parameter Setup ---
    if not (0 <= rho < 1):
        raise ValueError("Traffic intensity rho must be in the range [0, 1).")
    
    # --- 2. Determine the Truncation Point for the Distributions ---
    # The final support will be [0, 1, ..., max_k]
    
    # Condition 1: Truncate based on padding
    max_a = max(a) if len(a) > 0 else 0
    trunc_point_padding = max_a + padding
    
    # Condition 2: Truncate based on tail mass tolerance
    # We need to find N such that rho**(N+1) < tolerance
    if rho > 0:
        # (N+1) * log(rho) < log(tolerance) => N+1 > log(tolerance) / log(rho) (log(rho) is negative)
        trunc_point_tol = math.ceil(math.log(tolerance) / math.log(rho)) - 1
    else: # if rho is 0, the distribution is just P(0)=1
        trunc_point_tol = 0

    # The final truncation point is the larger of the two
    max_k = max(trunc_point_padding, trunc_point_tol)
    
    # The common support for both distributions
    support = np.arange(max_k + 1)
    
    # --- 3. Construct the Full PMF for the Empirical Distribution Q ---
    emp_weights = np.zeros(max_k + 1)
    for i in range(len(a)):
        if a[i] <= max_k:
            emp_weights[a[i]] = q[i]
            
    # Normalize just in case the original q was not normalized
    if emp_weights.sum() > 0:
        emp_weights /= emp_weights.sum()

    # --- 4. Construct the Full PMF for the Theoretical Distribution X ---
    # Calculate the geometric distribution probabilities
    th_weights = (1 - rho) * (rho ** support)
    
    # The tail mass is everything from max_k + 1 onwards.
    # For a geometric distribution, this sum is exactly rho**(max_k + 1)
    tail_mass = rho**(max_k + 1)
    
    # Add the tail mass to the last point to ensure the distribution sums to 1
    th_weights[max_k] += tail_mass
    
    # --- 5. Compute and Return the Wasserstein Distance ---
    return wasserstein_distance(support, support, u_weights=emp_weights, v_weights=th_weights)

### 2. Stein Discrepancies

In [3]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB

In [29]:
def SD_MM1_V1(q: list, a: list, lmd: float, mu: float):
    """
    Computes the W1 distance by solving a robust, sparse Linear Program.

    This version explicitly defines variables for both the slope (x_k) and
    the curvature (y_k) for maximum clarity and theoretical soundness.
    It handles unsorted input and ensures 0 and 1 are in the support.

    Parameters
    ----------
    q : list or np.ndarray
        Probability weights corresponding to 'a'.
    a : list or np.ndarray
        Support points. Does not need to be sorted.
    lmd : float
        Arrival rate for the M/M/1 queue.
    mu : float
        Service rate for the M/M/1 queue.

    Returns
    -------
    obj_val : float or None
        The optimal objective value (the W1 distance). None if not optimal.
    status : str
        The human-readable Gurobi solver status.
    """
    # --- 1. Input Validation & Data Pre-processing ---
    if len(q) != len(a):
        raise ValueError("Length of weights q and support a must be equal.")
    if mu <= lmd:
        raise ValueError("Service rate mu must be greater than arrival rate lmd.")

    point_map = {k: v for k, v in zip(a, q)}
    point_map.setdefault(0, 0.0)
    point_map.setdefault(1, 0.0)

    sorted_points = sorted(point_map.items())
    a = [point[0] for point in sorted_points]
    q = [point[1] for point in sorted_points]
    
    stability_bound_C = 1.0 / (mu - lmd)

    # --- 2. Identify all necessary "Critical Points" ---
    critical_points_base = set()
    for point in a:
        critical_points_base.add(point)
        if point > 0:
            critical_points_base.add(point - 1)
    
    if not critical_points_base:
        critical_points_base.add(0)

    # max_base_point = max(critical_points_base)
    # critical_points_x = critical_points_base.union({max_base_point + 1})
    critical_points_x = critical_points_base

    K_x = sorted(list(critical_points_x))
    print(K_x)
    p_x = len(K_x)
    x_point_to_idx = {k: j for j, k in enumerate(K_x)}

    K_y = [k for k in K_x if k + 1 in x_point_to_idx]
    p_y = len(K_y)
    y_point_to_idx = {k: j for j, k in enumerate(K_y)}
    print(y_point_to_idx)
    
    # --- 3. Build the Gurobi LP Model ---
    model = gp.Model("SD_MM1_V1")
    model.Params.OutputFlag = 0
    
    # --- 4. Define EXPLICIT Variables for x_k and y_k ---
    x = model.addVars(p_x, lb=-GRB.INFINITY, name="x")
    y = model.addVars(p_y, lb=-GRB.INFINITY, name="y")

    # --- 5. Set the Objective Function ---
    obj_expr = 0
    for i in range(len(a)):
        a_i, q_i = a[i], q[i]
        if q_i == 0: continue
        
        idx_ai = x_point_to_idx[a_i]
        obj_expr += q_i * lmd * x[idx_ai]
        
        if a_i > 0:
            idx_ai_minus_1 = x_point_to_idx[a_i - 1]
            obj_expr += q_i * (-mu) * x[idx_ai_minus_1]
            
    model.setObjective(obj_expr, GRB.MAXIMIZE)

    # --- 6. Set ALL Constraints Explicitly ---

    # 6a. Consistency Constraint: y_k = x_{k+1} - x_k
    for k_y_val, j_y in y_point_to_idx.items():
        j_x_curr = x_point_to_idx[k_y_val]
        j_x_next = x_point_to_idx[k_y_val + 1]
        model.addConstr(y[j_y] == x[j_x_next] - x[j_x_curr])

    # 6b. Stability Constraint: |y_k| <= C
    for j_y in range(p_y):
        model.addConstr(y[j_y] <= stability_bound_C)
        model.addConstr(y[j_y] >= -stability_bound_C)

    # 6c. Lipschitz-on-Generator Constraint: |lambda*y_k - mu*y_{k-1}| <= 1
    idx_y0 = y_point_to_idx[0]
    idx_x0 = x_point_to_idx[0]
    expr_init = lmd * y[idx_y0] - mu * x[idx_x0]
    model.addConstr(expr_init <= 1)
    model.addConstr(expr_init >= -1)
    
    for k_y_val, j_y_curr in y_point_to_idx.items():
        if k_y_val > 0 and (k_y_val - 1) in y_point_to_idx:
            j_y_prev = y_point_to_idx[k_y_val - 1]
            expr = lmd * y[j_y_curr] - mu * y[j_y_prev]
            model.addConstr(expr <= 1)
            model.addConstr(expr >= -1)

    # --- 7. Solve and Return ---
    model.optimize()
    
    # FIX: Map Gurobi status code to a human-readable string
    status_code = model.Status
    status_map = {
        GRB.OPTIMAL: "OPTIMAL",
        GRB.UNBOUNDED: "UNBOUNDED",
        GRB.INFEASIBLE: "INFEASIBLE",
        GRB.INF_OR_UNBD: "INF_OR_UNBOUNDED",
        GRB.TIME_LIMIT: "TIME_LIMIT",
    }
    status = status_map.get(status_code, f"UNKNOWN_STATUS_{status_code}")
    
    obj_val = -1
    if status_code == GRB.OPTIMAL:
        obj_val = model.ObjVal
        
    return obj_val, status

In [30]:
# --- Shared Parameters ---
lmd_val = 0.7
mu_val = 1.0
rho_val = lmd_val / mu_val

print(f"Testing with Parameters: lambda={lmd_val}, mu={mu_val} (rho={rho_val:.2f})")
print("="*60)

# --- Test Case 1: Dense Distribution ---
print("\n--- Test Case 1: Dense Distribution ---")
a_dense = [0, 1, 2, 3, 4]
q_dense = np.array([0.1, 0.4, 0.3, 0.1, 0.1])
q_dense /= q_dense.sum()
print(f"Support a = {a_dense}")

# Calculate using our sparse LP
lp_val_dense, status_dense = SD_MM1_V1(q_dense, a_dense, lmd_val, mu_val)
print(f"  SD_MM1_V1 (LP Solver) Result: {lp_val_dense:.8f} (Status: {status_dense})")

# Calculate using standard library
w1_val_dense = W1_MM1(q_dense, a_dense, rho_val)
print(f"  W1_MM1 (Scipy) Result:      {w1_val_dense:.8f}")

# Compare results
if lp_val_dense is not None:
    diff_dense = abs(lp_val_dense - w1_val_dense)
    print(f"  Absolute Difference:          {diff_dense:.8f}")


# --- Test Case 2: Sparse Distribution with a large gap ---
print("\n--- Test Case 2: Sparse Distribution ---")
a_sparse = [0, 1, 100, 101]
q_sparse = np.array([0.4, 0.3, 0.1, 0.2])
q_sparse /= q_sparse.sum()
print(f"Support a = {a_sparse}")

# Calculate using our sparse LP
lp_val_sparse, status_sparse = SD_MM1_V1(q_sparse, a_sparse, lmd_val, mu_val)
print(f"  SD_MM1_V1 (LP Solver) Result: {lp_val_sparse:.8f} (Status: {status_sparse})")

# Calculate using standard library
w1_val_sparse = W1_MM1(q_sparse, a_sparse, rho_val)
print(f"  W1_MM1 (Scipy) Result:      {w1_val_sparse:.8f}")

# Compare results
if lp_val_sparse is not None:
    diff_sparse = abs(lp_val_sparse - w1_val_sparse)
    print(f"  Absolute Difference:          {diff_sparse:.8f}")

Testing with Parameters: lambda=0.7, mu=1.0 (rho=0.70)

--- Test Case 1: Dense Distribution ---
Support a = [0, 1, 2, 3, 4]
[0, 1, 2, 3, 4]
{0: 0, 1: 1, 2: 2, 3: 3}
  SD_MM1_V1 (LP Solver) Result: 1.05333333 (Status: OPTIMAL)
  W1_MM1 (Scipy) Result:      1.05333333
  Absolute Difference:          0.00000000

--- Test Case 2: Sparse Distribution ---
Support a = [0, 1, 100, 101]
[0, 1, 99, 100, 101]
{0: 0, 99: 1, 100: 2}
  SD_MM1_V1 (LP Solver) Result: -1.00000000 (Status: INF_OR_UNBOUNDED)
  W1_MM1 (Scipy) Result:      28.83266667
  Absolute Difference:          29.83266667
